# L0.3 - Basic image processing

In this lab you will solve a small collection of image processing tasks. These are kept simple and should help you to understand some basic concepts as well as to get acquainted with the previously introduced libraries and tools.

Intended time: **max 1.5 hours**

Questions to: **daniel.jorgens@sth.kth.se**

### References
 1. http://scikit-image.org/docs/dev/: Documentation of _scikit-image_. Use the search functionality to find suitable functions for your purposes.
 2. http://insightsoftwareconsortium.github.io/SimpleITK-Notebooks/Python_html/03_Image_Details.html: Hints on how to load an image in _SimpleITK_.
 3. https://docs.scipy.org/doc/scipy/reference/ndimage.html: Documentation of _ndimage_ subpackage of _SciPy_.

### Helper functions

The following cell defines a function for convenient plotting that you can use for the subsequent tasks.

In [1]:
def plot_param_series(im_series, param_series, series_name, param_name):
    """
    Plot a series of images in one row. Include the corresponding parameter
    given to the function in the title. Name of image series and parameter
    name are always used in the same way ('<series_name> (<param_name> = <value>)').
    
    :param im_series: contains images to be plotted
    :type im_series: list of numpy arrays
    :param param_series: contains parameter values corresponding to the images in 'im_series'
    :type param_series: list of scalars
    :param series_name: see above
    :type series_name: str
    :param param_name: see above
    :type param_name: str
    :rtype: None
    """
    
    # plot results #
    plt.figure(figsize=(20, 8))

    plt_opts = dict(vmin=im_series[0].min(), vmax=im_series[0].max())
    
    for i, (im, param) in enumerate(zip(im_series, param_series)):
        ax = plt.subplot(1, len(im_series), i+1)
        ax.imshow(im, **plt_opts)
        ax.set_title("{} ({} = {})".format(series_name, param_name, param))

    plt.show()

In [2]:
def plot_sub_image(im, pos, title=None, size=(1, 5), plt_opts=dict()):
    """
    Add another axis object to subplot layout defined by given 'size' parameter.
    
    :param im: image to plot
    :type im: numpy array
    :param pos: position in subplot layout
    :type pos: integer
    :param title: title of image
    :type title: str
    :param size: number of rows and columns of subplot layout
    :type size: 2-tuple of integers
    :param plt_opts: additional options passed to pyplot's 'imshow' function
    :type plt_opts: dict
    :return: created Axes object
    :rtype: matplotlib.axes.Axes
    """
    
    ax = plt.subplot(size[0], size[1], pos)
    ax.imshow(im, **plt_opts)
    ax.set_title(title)
    
    return ax

# Histograms

Histograms are one way to analyse the content of an image.

#### Task 1.a
 * Load an image. Use a suitable function in _SimpleITK_.
 * Convert it to a numpy array.
 * Plot the image and the histogram. Use a suitable functions in _matplotlib_ for both.

Explain the shape of the histogram and explain its relation to the image! What happens if you vary the number of bins in the histogram?

In [3]:
import os
import SimpleITK as sitk

# TODO: use SimpleITK to load the image file 'image_head.dcm' #
filename = os.path.abspath('image_head.dcm')
im = sitk.ReadImage(filename)

# TODO: convert the ITK image object to a numpy array #
im_array = sitk.GetArrayFromImage(im)


RuntimeError: Exception thrown in SimpleITK ReadImage: C:\d\VS14-Win64-pkg\SimpleITK\Code\IO\src\sitkImageReaderBase.cxx:99:
sitk::ERROR: The file "C:\Users\Andrés\Desktop\image_head.dcm" does not exist.

In [ ]:
import matplotlib.pyplot as plt

from matplotlib import rcParams
rcParams['font.size'] = 20

# TODO: define number of bins for histogram #
nbins = 20

# create a new figure #
plt.figure(figsize=(20,8))

# original image #
plt.subplot(1, 2, 1)
# TODO: use matplotlib's function for showing an image #
plt.imshow(im_array)
plt.subplot(1, 2, 2)
# TODO: use matplotlib's histogram function; specify number of bins #
plt.hist(im_array,nbins)

# choose the colormap #
plt.set_cmap('gray')

# make plots visible #
plt.show()

# Convolution

The convolution of a function $f$ with a filter $k$ is defined as $\mathrm{C}_k[f] = \int_{\mathbb{R}^n} k(s)f(s-x) \, \mathrm{d^n}x$.
In case $k$ is separable this operation can be decomposed into $n$ convolutions with one-dimensional filters $k_1, ..., k_n$ for which $k(s) = k_1(s_1) \cdot \ldots \cdot k_n(s_n)$ where $s \in \mathbb{R}^n$.

In this section you will confirm this relationship.

#### Task 2

* Create a 1d kernel and derive a 2d kernel by computing the outer product of the 1d kernel with itself. Use _Numpy_ for that.
* Obtain $g_k := \mathrm{C}_k[f]$ and $g_{k_{1,2}} := \mathrm{C}_{k_2} \circ \mathrm{C}_{k_1}[f]$ by performing one 2d convolution and two 1d convolutions, respectively. Use _Scipy_ for the convolution.
* Compute the difference image of $g_k$ and $g_{k_{1,2}}$.
* Plot the original image, the two convolution results $g_k$ and $g_{k_{1,2}}$ as well as their difference image.

Are $g_k$ and $g_{k_{1,2}}$ identical? What kind of structures to you obtain with the chosen filter?

In [ ]:
import numpy as np
from scipy.ndimage.filters import # ... #  # TODO: imports #

# define convolution kernel #
k_1d = [-1,0,1]  # TODO: define a 1d kernel for computing the difference of subsequent values #
k_2d = np.outer(k_1d,k_1d)  # TODO: compute a 2d kernel from its 1d version (hint: outer product) #
print("This is the computed kernel: \n {}".format(k_2d))

# perform convolution #
im_conv = np.convolve(im_array,k_2d,mode='same')  # TODO: perform one 2d convolution; choose a border mode #
im_conv12 = np.convolve(np.convolve(im_array,k_1d,mode='same'),k_1d.T,mode='same')  # TODO: perform two 1d convolutions

# compute difference image #
im_diff = im_conv12-im_conv  # TODO: compute difference #

In [ ]:
# plot results #
plt.figure(figsize=(20, 8))

plt_sz = (1, 4)
plt_opts = dict(vmin=im_conv.min(), vmax=im_conv.max())

plot_sub_image(im_array,  1, title="Original",            size=plt_sz)
plot_sub_image(im_conv,   2, title="$g_k$",               size=plt_sz, plt_opts=plt_opts)
plot_sub_image(im_conv12, 3, title="$g_{k_{1,2}}$",       size=plt_sz, plt_opts=plt_opts)
plot_sub_image(im_diff,   4, title="$g_k - g_{k_{1,2}}$", size=plt_sz, plt_opts=plt_opts)

plt.show()

# Image filtering

The choice of convolution kernel determines the kind of features visible in the output image. Choosing the famous Gaussian kernel results in a blurred output image.

#### Task 3.a

* Use gaussian filtering to blur an image. There is a _Scipy_ function for applying a gaussian filter directly.
* Vary the gaussian standard deviation $\sigma$.

What is the effect of increasing $\sigma$?

In [ ]:
from scipy.ndimage.filters import gaussian_filter  # TODO: imports #

# define series of standard devs for gaussian kernel #
sigma_list = range(5)

im_gauss_list = list()
for sigma in sigma_list:
    im_gauss = gaussian_filter(im_array,sigma)  # TODO: apply gaussian filter to image; use given 'sigma' #
    im_gauss_list.append(im_gauss)

In [ ]:
# plot results #
plot_param_series(im_gauss_list, sigma_list, 'Gauss', '$\sigma$')

#### Task 3.b

In the lecture it is mentioned that repeated filtering with a gaussian kernel with std dev $\sigma_1$ is equivalent to filtering with a kernel with std dev $\sigma_2 > \sigma_1$. Investigate this claim!

* Apply a gaussian filter with fixed $\sigma$ repeatedly.
* Choose a filtering reference from **Task 3.a** obtained with a single filtering pass ($\sigma_r$).
* Compute the difference image of that reference with each of the filtering repetitions.
* Plot the results from repeated filtering as well as the corresponding difference images.

What do you observe? What number of repetitions are needed to obtain the same result with repeated filtering? Vary the choices of $\sigma_r$ and $\sigma_{rep}$!

In [ ]:
# define repetitions for gaussian filtering #
rep_list = range(5)

# choose gaussian std dev #
sigma_rep = 3  # TODO: vary gaussian sigma for repeated filtering #

im_gauss_rep_list = list()
im_gauss_rep = im_array  # used for recursive implementation #
for _ in rep_list:
    im_gauss_rep = gaussian_filter(im_gauss_rep,sigma_rep)  # TODO: apply gaussian filter to image #
    im_gauss_rep_list.append(im_gauss_rep)

In [ ]:
# choose a directly gaussian filtered image as reference #
sigma_ref = 2  # TODO: vary the reference image #
im_gauss_reference = im_gauss_list[sigma_ref]

# plot results #
print("Repeating gaussian filtering (sigma = {}) ... ".format(sigma_rep))

plot_param_series(im_gauss_rep_list, rep_list, 'Mean', 'reps')

# compute list of difference images #
diff_gauss_rep_list = list(map(lambda x: x - im_gauss_reference, im_gauss_rep_list))
plot_param_series(diff_gauss_rep_list, [sigma_ref]*len(diff_gauss_rep_list), 'diff', '$\sigma_r$')

#### Task 3.c

In the lecture it is mentioned that repeated mean filtering can give similar results as gaussian filtering. Check this claim!

* Define a mean kernel. Use a _Numpy_ array for that purpose.
* Apply the mean filter repeatedly to the image as in **Task 3.b**. You may use the same approach as in **Task 3.a**.
* Pick a reference image as in **Task 3.b** and compute the difference images for all repetitions.
* Plot the resulting filtered images as well as the difference images.

What do you observe? How many repetitions are needed to obtain a similar result as with gaussian filtering? Vary the choice of $\sigma_r$!

In [ ]:
from scipy.ndimage.filters import uniform_filter
# define repetitions for mean filtering #
rep_list = range(5)

# TODO: define a mean filter in 2D #
mean_kernel = np.full((3,3),1/9)  # TODO: define 3x3 mean filter using 'numpy.full' #

im_mean_list = list()
im_mean = im_array  # used for recursive implementation #
for _ in rep_list:
    im_mean = uniform_filter(im_mean)  # TODO: apply mean filter to image #
    im_mean_list.append(im_mean)

In [ ]:
# choose a gaussian filtered image as reference #
sigma_ref = 2  # TODO: vary the reference image #
im_gauss_reference = im_gauss_list[sigma_ref]

# plot results #
print("Repeating mean filtering ... ")

plot_param_series(im_mean_list, rep_list, '', 'repetitions')

# create list of difference images #
diff_mean_list = list(map(lambda x: x - im_gauss_reference, im_mean_list))
plot_param_series(diff_mean_list, [sigma_ref]*len(diff_mean_list), 'diff', '$\sigma$')

# Edge detection

Filters that provide an approximation of the image gradient can be used for edge detection. Here, you will compare different of such filter kernels.

#### Task 4.a

* Apply the two (omit finite differences, you have used them already ;) ) filters for approximation of the image gradient magnitude mentioned in the lecture. You find suitable implementations in _scikit-image_.
* Apply the filters after gaussian smoothing. Reuse results from the previous tasks.
* Plot and compare the results.

Are there differences between the two filters? Do both have the same level of rotational variance? What is the effect of the gaussian smoothing prior to the filtering?

In [ ]:
from skimage.filters import sobel  # TODO: imports #
from skimage.filters import prewitt
# apply the filters to 'im_array' #
im_sobel = sobel(im_array)  # TODO: apply sobel filter #
im_prewitt = prewitt(im_array)  # TODO: apply prewitt filter #

In [ ]:
# pick gaussian filtered image as reference #
sigma_edge = 1  # TODO: vary 'sigma_edge' #
im_gauss_ref_edge = im_gauss_list[sigma_edge]

# apply filters to the chosen, gaussian-filtered reference image #
im_gauss_sobel = sobel(im_gauss)  # TODO: apply sobel filter #
im_gauss_prewitt = prewitt(im_gauss)  # TODO: apply prewitt filter #

In [ ]:
# plot results #
plt.figure(figsize=(20, 16))

plt_sz = (3, 4)

plot_sub_image(im_array,   1, title="Original", size=plt_sz)
plot_sub_image(im_sobel,   2, title="Sobel",    size=plt_sz, plt_opts=dict(vmin=0, vmax=im_sobel.max()))
plot_sub_image(im_prewitt, 3, title="Prewitt",  size=plt_sz, plt_opts=dict(vmin=0, vmax=im_sobel.max()))

diff_sobel_prewitt = im_sobel - im_prewitt
plot_sub_image(diff_sobel_prewitt, 4, title="{:5.4f} / {:5.4f}".format(diff_sobel_prewitt.min(),
                                                                       diff_sobel_prewitt.max()),
               size=plt_sz)

plot_sub_image(im_gauss_ref_edge,  5, title="Original ($\sigma = {}$)".format(sigma_edge), size=plt_sz)
plot_sub_image(im_gauss_sobel,     6, title="Sobel ($\sigma = {}$)".format(sigma_edge),    size=plt_sz,
               plt_opts=dict(vmin=0, vmax=im_gauss_sobel.max()))
plot_sub_image(im_gauss_prewitt,   7, title="Prewitt ($\sigma = {}$)".format(sigma_edge),  size=plt_sz,
               plt_opts=dict(vmin=0, vmax=im_gauss_sobel.max()))

diff_gauss_sobel_prewitt = im_gauss_sobel - im_gauss_prewitt
plot_sub_image(diff_gauss_sobel_prewitt, 8, title="{:5.4f} / {:5.4f}".format(diff_gauss_sobel_prewitt.min(),
                                                                             diff_gauss_sobel_prewitt.max()),
               size=plt_sz)

diff_orig = im_array - im_gauss_ref_edge
plot_sub_image(diff_orig,     9, title="{:5.1f} / {:5.1f}".format(diff_orig.min(), diff_orig.max()),       size=plt_sz)

diff_sobel = im_sobel - im_gauss_sobel
plot_sub_image(diff_sobel,   10, title="{:5.4f} / {:5.4f}".format(diff_sobel.min(), diff_sobel.max()),     size=plt_sz)

diff_prewitt = im_prewitt - im_gauss_prewitt
plot_sub_image(diff_prewitt, 11, title="{:5.4f} / {:5.4f}".format(diff_prewitt.min(), diff_prewitt.max()), size=plt_sz)

diff_diffs = diff_sobel_prewitt - diff_gauss_sobel_prewitt 
plot_sub_image(diff_diffs,   12, title="{:5.4f} / {:5.4f}".format(diff_diffs.min(), diff_diffs.max()),     size=plt_sz)

plt.show()

#### Task 4.b

* Apply the Canny edge detector to the input image. You find an implementation in _scikit-image_.
* Plot the result for several values of $\sigma_{canny}$.
* Vary the edge detectors parameters in order to obtain some result (in the standard setting nothing will appear).

Do you manage to tune the detector in order to obtain any reasonable result? Which values to choose for its three parameters? How do they work (hint: have a look at the documentation)? What is the difference between the Canny edge detector and the filters used in **Task 4.a**?

In [ ]:
from skimage.feature import canny  # TODO: imports #

# define list of sigma values used in Canny edge detector #
sigma_canny_list = range(5)

im_canny_list = list()
for sigma_canny in sigma_canny_list:
    # TODO: perform canny edge detection; specify 'sigma' and both thresholds #
    im_canny = canny(im_array,sigma_canny,50,100)  # TODO: vary thresholds #
    im_canny_list.append(im_canny)

In [ ]:
# plot results #
plot_param_series(im_canny_list, sigma_canny_list, 'Canny', '$\sigma$')

# Resampling

The process of changing the resolution of an image is also referred to as resampling. In case the target resolution is higher than the original one (i.e. the target image will have more pixels than the original), typically some kind of interpolation is involved.

#### Task 5

* Define a subregion of the original image to investigate in this task.
* Apply the three different kinds of interpolation that were discussed in the lecture.
* Vary interpolation order and magnification factor.
* Measure the computation time for each approach. Use the notebook magic _%time_.
* Plot the results alongside each other.

Are the results as you expected? Why? Which approach is better? Which is faster?

In [ ]:
from scipy.ndimage import zoom  # TODO: imports #

# TODO: define list of interpolation orders #
order_list = list(range(3))

# define a subregion of the image (to see more details) #
im_array_tile = im_array[40:60, 40:60]

im_interp_list = list()
for order_interp in order_list:
    # TODO: resample the image; specify the magnification factor and interpolation order #
    %time 
    z=2
    im_interp=zoom(im_array_tile,z,order_interp)
    # TODO: vary the magnification factor #
    im_interp_list.append(im_interp)

In [ ]:
# plot results #
plot_param_series(im_interp_list, order_list, 'Interp', 'order')

# create list of difference images #
interp_diff_list = list(map(lambda x: x - im_interp_list[-1], im_interp_list))
plot_param_series(interp_diff_list, order_list, 'Diff', 'order')

# Mathematical morphology

The morphological operations that were discussed in the lecture are a way to explore the shape of structures contained in the image.

#### Task 6.a

* Define a structuring element. You find predefined ones in _scikit-image_.
* Apply all four operations discussed in the lecture to our test image. Their implementations can be found in _scikit-image_.
* Plot the original image and the four results.

Compare the effect of different structuring elements. Is there a difference in the result of the morphological operation? What happens if you change the size of the structuring element?

In [ ]:
from skimage.morphology import disk,dilation,erosion,closing,opening  # TODO: imports #

# TODO: define structuring element (e.g. a disk of size 2) #
selem = disk(2)  # TODO: vary size and type of element #

# TODO: apply the four morphological operations #
im_erode = erosion(im_array,selem)
im_dilate = dilation(im_array,selem)
im_open = opening(im_array,selem)
im_close = closing(im_array,selem)

In [ ]:
# plot results #
plt.figure(figsize=(20, 8))

plt_sz = (1, 5)

plot_sub_image(im_array,  1, title="Original", size=plt_sz)
plot_sub_image(im_erode,  2, title="Erosion",  size=plt_sz)
plot_sub_image(im_dilate, 3, title="Dilation", size=plt_sz)
plot_sub_image(im_open,   4, title="Opening",  size=plt_sz)
plot_sub_image(im_close,  5, title="Closing",  size=plt_sz)

plt.show()

#### Task 6.b

* Apply the operations *white_tophat* and *black_tophat* to the test image. The functions can be found in _scikit-image_.
* Plot the original image and the two filtering results.

What do these operations do? Explain the two filtering outputs!

In [ ]:
from skimage.morphology import white_tophat,black_tophat,square,rectangle,diamond  # TODO: imports #

# TODO: define structuring element #
selem = square(3)  # TODO: vary size and type of element #

im_white_top = white_tophat(im_array,selem)  # TODO: apply 'white_tophat' operation #
im_black_top = black_tophat(im_array,selem)  # TODO: apply 'black_tophat' operation #

In [ ]:
# plot results #
plt.figure(figsize=(20, 8))

plt_sz = (1, 3)

plot_sub_image(im_array,     1, title="Original",      size=plt_sz)
plot_sub_image(im_white_top, 2, title="White top hat", size=plt_sz)
plot_sub_image(im_black_top, 3, title="Black top hat", size=plt_sz)

plt.show()

# Distance transform

A distance transform computes for each pixel in a mask the distance to the closest point on the border of that mask.

#### Task 7

* Create a mask for the test image e.g. by simple thresholding. Use a numpy array to store it.
* Compute the distance transform for that mask. _Scipy_ provides an implementation for that.
* Plot the original image and the resulting transform.

Does the result make sense? Explain what you see!

In [ ]:
from scipy.ndimage.morphology import distance_transform_edt as distance  # TODO: imports #

# TODO: define image mask (hint: thresholding on image array) #
im_mask = im_array>50

# TODO: apply distance transform on the mask #
im_edt = distance(im_mask)

In [ ]:
# plot results #
plt.figure(figsize=(20, 8))

plt_sz = (1, 3)

plot_sub_image(im_array, 1, title="Original", size=plt_sz)
plot_sub_image(im_mask,  2, title="Mask",     size=plt_sz)
plot_sub_image(im_edt,   3, title="dist xfm", size=plt_sz)

plt.show()

# Congratulations!

You made it to the end! :)

Tell your lab assistant about your results and you will pass this lab!